# 验证数据集和交叉验证
> Validation 和 Cross Validation

## 为了防止我们仅针对测试数据集做优化，方向会被带偏，所以我们有分出了一部分数据作为验证集
![测试数据集和验证数据集的作用](images/测试数据集和验证数据集的作用.png)
## 为了防止随机数据导致的不确定性，训练集、验证集、测试集往往需要调换身份，即交叉验证
![交叉验证](images/交叉验证.png)

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

### 测试train_test_split

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=666)

In [4]:
from sklearn.neighbors import KNeighborsClassifier

best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k, best_p, best_score = k, p, score
            
print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score)

Best K = 3
Best P = 4
Best Score = 0.9860917941585535


## 使用交叉验证

In [5]:
from sklearn.model_selection import cross_val_score # sklearn集成的交叉验证方法

knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train, cv=3)

array([0.98895028, 0.97777778, 0.96629213])

In [6]:
best_k, best_p, best_score = 0, 0, 0
for k in range(2, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=k, p=p)
        scores = cross_val_score(knn_clf, X_train, y_train, cv=3)
        score = np.mean(scores)
        if score > best_score:
            best_k, best_p, best_score = k, p, score
            
print("Best K =", best_k)
print("Best P =", best_p)
print("Best Score =", best_score) # 交叉验证得到的结果和上面非交叉验证得到的k和p不同，我们更相信交叉验证的

Best K = 2
Best P = 2
Best Score = 0.9823599874006478


In [7]:
best_knn_clf = KNeighborsClassifier(weights="distance", n_neighbors=2, p=2) # 参数详解见 https://www.cnblogs.com/pinard/p/6065607.html
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.980528511821975

## 回顾网格搜索
> GridSearchCV本身内部已经实现了交叉验证

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(2, 11)], 
        'p': [i for i in range(1, 6)]
    }
]

grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


D:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:   23.3s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=10, p=5,
                                            weights='distance'),
             iid='warn', n_jobs=None,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [9]:
grid_search.best_score_ # 最佳score

0.9823747680890538

In [10]:
grid_search.best_params_ # 最优参数

{'n_neighbors': 2, 'p': 2, 'weights': 'distance'}

In [11]:
best_knn_clf = grid_search.best_estimator_
best_knn_clf.score(X_test, y_test)

0.980528511821975

### cv参数
> 训练数据集分成几份，默认是3份

In [12]:
cross_val_score(knn_clf, X_train, y_train, cv=5) # cv=5就返回了最佳的5个模型

array([0.99543379, 0.96803653, 0.98148148, 0.96261682, 0.97619048])

In [13]:
grid_search = GridSearchCV(knn_clf, param_grid, verbose=1, cv=5)

## 交叉验证的常见方法

### k-folds交叉验证
> 把训练数据集分成k份，称为k-folds cross validation

缺点：每次训练k个模型，相当于整体性能满了k倍

### 留一法 LOO-CV
> 把训练数据集分成m份，称为留一法 Leave-One-Out Cross Validation

+ 优点：完全不受随机的影响，最接近模型真正的性能指标
+ 缺点：计算量巨大